In [92]:
import sys
import os
import numpy as np
import mne
import matplotlib.pyplot as plt

from mne.preprocessing import ICA
from pyprep.find_noisy_channels import NoisyChannels

from autoreject import AutoReject
from autoreject import get_rejection_threshold

from IPython import display

from scipy.io import savemat

## define label dictionary for epoch annotations
custom_mapping = {'eyes open': 2, 'pinprick hand': 3,'plastic hand': 5, 'pinprick back': 6, 
                  'plastic back': 8,'stop': 9, # original naming convention (subjs <20)
                  'eyes closed': 1, 'eyes opened': 2, 'Yes Pain Hand': 3, # newer naming convention (subs <20)
                  'Med Pain Hand': 4,  'No Pain Hand': 5, 'Yes Pain Back': 6, 
                  'Med Pain Back': 7, 'No Pain Back': 8, 'Stop': 9, 
                  '1000001': 10, '1000010': 11, '1100001': 12, '1100010': 13, # 65, 66, 97, 98
                  }

## define functions for extracting relevant epochs
def delete_multiple_elements(list_object, indices):
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)

def get_stim_epochs(epochs,events_from_annot,event_dict):
    # get lists for keys and values of event_dict
    key_list = list(event_dict.keys())
    val_list = list(event_dict.values())

    # intialize StimOn_ids for epoch indices
    StimOn_ids = []
    for i in range(len(epochs)-1):
        # current epoch
        curr_pos = val_list.index(events_from_annot[i][-1]) # get position of epoch description value 
        curr_key_str = key_list[curr_pos] # get key at position (e.g., 'Yes Pain Hand')

        # next epoch
        next_pos = val_list.index(events_from_annot[i+1][-1]) # get position of epoch description value 
        next_key_str = key_list[next_pos] # get key at position (e.g., '1100001')

        if ( curr_key_str[0].isalpha() and not next_key_str[0].isalpha() ):
            StimOn_ids.append(i+1)
    return StimOn_ids


In [94]:
# subject ID
data_dir = "../../../Matlab Code/Processed Data/"

# file = '040_preprocessed-raw.fif'
for file in os.listdir(data_dir):
    if not (os.path.isdir(file) and file.startswith('.')):
        # read data, set EOG channel, and drop unused channels
        sub_id = file
        print(f"{sub_id}\nreading preprocessed-raw file...")
        raw = mne.io.read_raw_fif(data_dir+file, preload=True)
        # raw = mne.io.read_raw_fif(data_dir+sub_num+'_preprocessed-raw.fif', preload=True)

        # create events to epochize data
        print(f"{sub_id}\ncreating events and epochizing data...")
        (events_from_annot, event_dict) = mne.events_from_annotations(raw, event_id=custom_mapping)
        epochs = mne.Epochs(raw, events_from_annot, event_dict, tmin=-0.2, tmax=0.8, proj=True, preload=True,
                            event_repeated='drop')#, baseline=(None,0))
        # display.clear_output(wait=True)

        # del raw # clear memory
        
        ### Create drop_logs for newer convention only, for now.
        if 'Yes Pain Hand' in raw.annotations.description and len(events_from_annot) > 100:            
            # adjust events_from_annot for dropped events
            print(f"{sub_id}\nremoving repeated epochs from annotations...")
            epochs_drop_log = list(epochs.drop_log)
            epochs_drop_ids = [idx for idx, tup in enumerate(epochs_drop_log) if epochs_drop_log[idx]]
            events_from_annot_list = events_from_annot.tolist()
            delete_multiple_elements(events_from_annot_list, epochs_drop_ids)
            events_from_annot_new = np.array(events_from_annot_list)
            # display.clear_output(wait=True)

            # find epochs only from stim events
            print(f"{sub_id}\nfinding the 60 pin-prick epochs...")
            StimOn_ids = get_stim_epochs(epochs,events_from_annot_new,event_dict)
            stim_epochs = epochs[StimOn_ids]
            # display.clear_output(wait=True)

            # use autoreject package to automatically clean epochs 
            print(f"{sub_id}\nfinding epochs to clean...")
            ar = AutoReject(random_state=42)
            _, reject_log = ar.fit_transform(stim_epochs, return_log=True)
            # display.clear_output(wait=True)

            del epochs, stim_epochs

            bad_epochs_bool = reject_log.bad_epochs.tolist()
            dropped_epochs_list = [i for i, val in enumerate(bad_epochs_bool) if val]
            print(dropped_epochs_list)

            # save copy of data
            print("saving bad_epochs as mat file...")
            mdic = {"drop_log": dropped_epochs_list}
            save_dir = "../../../Matlab Code/Drop Logs/"
            savemat(save_dir+file[:3]+'_drop_log.mat', mdic)

            print("Done.")
            display.clear_output(wait=True)

            # break
        # break
    # break

020_preprocessed-raw.fif
reading preprocessed-raw file...
Opening raw data file ../../../Matlab Code/Processed Data/020_preprocessed-raw.fif...
    Range : 0 ... 928639 =      0.000 ...  2321.597 secs
Ready.
Reading 0 ... 928639  =      0.000 ...  2321.597 secs...
020_preprocessed-raw.fif
creating events and epochizing data...


ValueError: Could not find any of the events you specified.

In [95]:
len(events_from_annot)

733

In [98]:
event_dict

{'1000001': 10,
 '1000010': 11,
 '1100001': 12,
 '1100010': 13,
 'Med Pain Back': 7,
 'Med Pain Hand': 4,
 'No Pain Back': 8,
 'No Pain Hand': 5,
 'Stop': 9,
 'Yes Pain Back': 6,
 'Yes Pain Hand': 3,
 'eyes closed': 1,
 'eyes opened': 2}

In [100]:
raw.annotations.description

array(['100016', '100032', '100048', '100048', '100048', '100048',
       '100048', '100048', '100048', '100048', '100016', '100048',
       '100048', '100048', '100048', '100048', '100032', '100048',
       '100016', '100048', '100048', '100048', '100048', '100048',
       '100048', '100048', '100048', '100016', '100032', '100048',
       '100048', '100016', '100048', '100048', '100048', '100016',
       '100048', '100048', '100048', '100048', '100016', '100016',
       '100048', '100016', '100048', '100032', '100016', '100032',
       '100048', '100048', '100048', '100048', '100016', '100048',
       '100048', '100048', '100048', '100048', '100048', '100048',
       '100016', '100048', '100048', '100048', '100048', '100048',
       '100032', '100048', '100048', '100048', '100048', '100048',
       '100048', '100048', '100048', '100032', '100032', '100048',
       '100032', '100048', '100032', '100048', '100032', '100032',
       '100048', '100048', '100016', '100032', '100048', '1000

In [109]:
# subject ID
data_dir = "../../../Matlab Code/Processed Data/"
epo_dir = "../../../Matlab Code/Processed Epoch Data/"

file = '040_preprocessed-raw.fif'
# for file in os.listdir(data_dir):
if not (os.path.isdir(file) and file.startswith('.')):
    # read data, set EOG channel, and drop unused channels
    sub_id = file
    print(f"{sub_id}\nreading preprocessed-raw file...")
    raw = mne.io.read_raw_fif(data_dir+file, preload=True)
    # raw = mne.io.read_raw_fif(data_dir+sub_num+'_preprocessed-raw.fif', preload=True)

    # create events to epochize data
    print(f"{sub_id}\ncreating events and epochizing data...")
    (events_from_annot, event_dict) = mne.events_from_annotations(raw, event_id=custom_mapping)
    epochs = mne.Epochs(raw, events_from_annot, event_dict, tmin=-0.2, tmax=0.8, proj=True, preload=True,
                        event_repeated='drop')#, baseline=(None,0))
    # display.clear_output(wait=True)

    # del raw # clear memory

    if 'Yes Pain Hand' in raw.annotations.description:
        # adjust events_from_annot for dropped events
        print(f"{sub_id}\nremoving repeated epochs from annotations...")
        epochs_drop_log = list(epochs.drop_log)
        epochs_drop_ids = [idx for idx, tup in enumerate(epochs_drop_log) if epochs_drop_log[idx]]
        events_from_annot_list = events_from_annot.tolist()
        delete_multiple_elements(events_from_annot_list, epochs_drop_ids)
        events_from_annot_new = np.array(events_from_annot_list)
        # display.clear_output(wait=True)

        # find epochs only from stim events
        print(f"{sub_id}\nfinding the 60 pin-prick epochs...")
        StimOn_ids = get_stim_epochs(epochs,events_from_annot_new,event_dict)
        stim_epochs = epochs[StimOn_ids]
        # display.clear_output(wait=True)

        # use autoreject package to automatically clean epochs 
        print(f"{sub_id}\nfinding epochs to clean...")
        ar = AutoReject(random_state=42)
        _, reject_log = ar.fit_transform(stim_epochs, return_log=True)
        # display.clear_output(wait=True)
        
        del epochs, stim_epochs

        bad_epochs_bool = reject_log.bad_epochs.tolist()
        dropped_epochs_list = [i for i, val in enumerate(bad_epochs_bool) if val]
        print(dropped_epochs_list)

        # save copy of data
        print("saving bad_epochs as mat file...")
        mdic = {"drop_log": dropped_epochs_list}
        savemat(data_dir+file[:3]+'_drop_log.mat', mdic)

        print("Done.")
        display.clear_output(wait=True)

        # break
    # break
# break

040_preprocessed-raw.fif
reading preprocessed-raw file...
Opening raw data file ../../../Matlab Code/Processed Data/040_preprocessed-raw.fif...
    Range : 0 ... 535839 =      0.000 ...  1339.598 secs
Ready.
Reading 0 ... 535839  =      0.000 ...  1339.598 secs...
040_preprocessed-raw.fif
creating events and epochizing data...
Used Annotations descriptions: ['1000001', '1000010', '1100001', '1100010', 'Med Pain Back', 'Med Pain Hand', 'No Pain Back', 'No Pain Hand', 'Stop', 'Yes Pain Back', 'Yes Pain Hand', 'eyes closed', 'eyes opened']
Multiple event values for single event times found. Keeping the first occurrence and dropping all others.
Not setting metadata
533 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 533 events and 401 original time points ...
0 bad epochs dropped
040_preprocessed-raw.fif
removing repeated epochs from annotations...
040_preprocessed-ra

/home/sb10flpc002/anaconda3/envs/eeg_env/lib/python3.7/site-packages/autoreject/utils.py:69: UserWarning: 2 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  'remove them from epochs.info["bads"].' % n_bads)


  0%|          | Creating augmented epochs : 0/62 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/62 [00:00<?,       ?it/s]

KeyboardInterrupt: 

In [ ]:
len(events_from_annot)

In [ ]:
events_from_annot

In [ ]:
event_dict

In [ ]:
raw.annotations.description

In [103]:
import eeglabio
mne.export.export_raw(data_dir+'040'+'_preprocessed-raw.set', raw, overwrite=True, verbose=True)
mne.export.export_epochs(data_dir+file[:3]+'_preprocessed-epo.set', epochs, overwrite=True, verbose=True)

In [108]:
epo_dir = "../../../Matlab Code/Processed Epoch Data/"
stim_epochs = mne.read_epochs(epo_dir+'040'+'_preprocessed-epo.fif')
mne.export.export_epochs(epo_dir+'040'+'_preprocessed-epo.set', stim_epochs, overwrite=True, verbose=True)

Reading /home/sb10flpc002/Documents/George Kenefati/EEG Decoding Project/Python code/Preprocessing/Pipeline (for deployment)/../../../Matlab Code/Processed Epoch Data/040_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
60 matching events found
No baseline correction applied
0 projection items activated
